In [2]:
import os
import tempfile
import pandas as pd
import random
import boto3
from langchain_community.llms import Bedrock
from langchain_aws import BedrockLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import time
import certifi
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# Set the AWS profile
os.environ["AWS_PROFILE"] = "default"
os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()

# Create a temporary directory for storing the uploaded files
temp_dir = tempfile.TemporaryDirectory()

# Function to validate the uploaded file
def validate_file(file_path):
    try:
        # Read the file using pandas
        df = pd.read_excel(file_path)
        return True, df.shape[0]
    except Exception as e:
        return False, 0

# Create the Bedrock client
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)

# Set the model ID
model_id = "meta.llama3-8b-instruct-v1:0"

# Create the Bedrock instance
llm = BedrockLLM(
    model_id=model_id,
    client=bedrock_client,
    model_kwargs={"max_gen_len": 512, "temperature": 0.5}
)

# Define the chat function
def chat(question):
    prompt = PromptTemplate(
        input_variables=["question"],
        template="{question}"
    )

    bedrock_chain = LLMChain(llm=llm, prompt=prompt)

    response = bedrock_chain({'question': question})
    return response

# Placeholder for user input file paths
users_file_path = "users.xlsx"  # Replace with the actual path
questions_file_path = "questions.xlsx"  # Replace with the actual path

# Validate the files
is_users_valid, user_count = validate_file(users_file_path)
is_questions_valid, question_count = validate_file(questions_file_path)

error_log = []

if is_users_valid and is_questions_valid:
    # Read the user details and questions from the files
    users = pd.read_excel(users_file_path)
    questions = pd.read_excel(questions_file_path)["Question"].tolist()

    user_progress = {}  # Dictionary to store the progress for each user
    user_time = {}  # Dictionary to store the total time taken for each user

    # Simulate volume testing
    for user_id in users.index:
        user_progress[user_id] = {"answered": 0, "error": 0}  # Initialize progress for each user
        user_time[user_id] = 0  # Initialize total time taken for each user
        random.shuffle(questions)  # Shuffle the questions for each user

        for i, question in enumerate(questions):
            try:
                # Start the timer
                start_time = time.time()

                # Get the answer from the model
                response = chat(question)
                answer = response["text"]

                # Stop the timer
                end_time = time.time()
                time_taken = end_time - start_time
                user_time[user_id] += time_taken

                user_progress[user_id]["answered"] += 1  # Increment the total answered questions
            except Exception as e:
                error_info = f"Error occurred for user {user_id} question: {question}. Error: {str(e)}"
                error_log.append(error_info)
                user_progress[user_id]["error"] += 1  # Increment the total error count

            # Create a DataFrame from the user_progress dictionary
            progress_df = pd.DataFrame(user_progress).T

            # Plot the progress for each user
            fig, ax = plt.subplots(figsize=(10, 5))  # Set a reasonable size for the plot
            progress_df.plot(kind='bar', stacked=True, color=['green', 'red'], ax=ax)
            ax.set_xlabel('User ID')
            ax.set_ylabel('Number of Questions')
            ax.set_title('Progress of Answered Questions for Each User')
            ax.yaxis.get_major_locator().set_params(integer=True)  # Ensure y-axis has integer values only

            # Display time taken on each bar
            for p in ax.patches:
                user_id = int(p.get_x() + p.get_width() / 2)
                time_text = f"time: {user_time[user_id] / 60:.2f} min"
                ax.annotate(time_text, (p.get_x() + p.get_width() / 2, p.get_height() + 0.5), 
                            ha='center', va='center', fontsize=10, color='black', fontweight='bold')

            clear_output(wait=True)
            display(fig)
            plt.close(fig)

    print(f"Volume testing completed with {user_count} users and {question_count} questions.")
    if error_log:
        print("Errors occurred during volume testing. Here are the details:")
        for error in error_log:
            print(error)
else:
    print("One or both of the uploaded files are not valid Excel files.")





One or both of the uploaded files are not valid Excel files.
